# Deploy Specific Model Version to Red Hat OpenShift Data Science

This notebook deploys the model.joblib file to OpenShift Data Science through Inference Service.
It is required that the model.joblib and inference-service.yaml file exists.
In the pipeline, the model.joblib file is generated by the preceding job.

In [2]:
import os, time, subprocess
from jinja2 import Template

In [7]:
storage_key="aws-connection-fraud-detection"
model_version=os.environ.get("model_version", "24-09-23-200017").replace("model-","")
model_name = 'fraud'

template_data = {"model_version": model_version, "storage_key": storage_key, "model_name": model_name}

def deploy_template(filename, template_data):
    print("invoking template:" + filename)
    template = Template(open(filename).read())
    rendered_template = template.render(template_data)

    subprocess.run(['oc', 'whoami'])
    ps = subprocess.Popen(['echo', rendered_template], stdout=subprocess.PIPE)
    print(ps.stdout)
    output = subprocess.check_output(['oc', 'apply', '-f', '-'], stdin=ps.stdout)
    ps.wait()
    print(f"Deployed template {filename}. Version: {model_version}")

templates = ['deployment/inference-service.yaml']
for t in templates:
    deploy_template(t, template_data)

invoking template:deployment/inference-service.yaml
system:serviceaccount:worldline:fraud-detection-wb
<_io.BufferedReader name=63>
Deployed template deployment/inference-service.yaml. Version: 24-09-23-200017
